In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
import pickle
import requests
import numpy as np
import pandas as pd
import time
import csv
import re
import json
import xlrd
import os
from selenium.common.exceptions import NoSuchElementException
from datetime import datetime, date, timedelta
import nltk
#from flatten_json import flatten
regex = re.compile(r"\[|\]|<", re.IGNORECASE)
import time
import copy

In [4]:
%cd /home/stu/code/dans

/home/stu/code/dans


In [ ]:
#
### Dans
#make headless
#options = Options()
#options.headless = True

#driver = webdriver.Firefox(options=options,executable_path=r'/usr/local/bin/geckodriver')

#path = "https://www.danmurphys.com.au/red-wine/all"
#driver.get(path)   

#file = "scraped_" + time.strftime("%Y%m%d") + ".csv"


# with open(file, "w") as csvFile:
#     #path = "https://www.danmurphys.com.au/red-wine/all"
#     fieldnames = ['URL']
#     writer = csv.DictWriter(csvFile, fieldnames=fieldnames)
#     writer.writeheader()
#     for item in range(1,223):
#         driver = webdriver.Firefox(executable_path=r'/usr/local/bin/geckodriver')
#         path = "https://www.danmurphys.com.au/red-wine/all?page=" + str(item)
#         driver.implicitly_wait(1)
#         driver.get(path)
#         elems = driver.find_elements_by_xpath("//a[@href]")
#         for elem in elems:
#             writer.writerow({'URL': elem.get_attribute("href")})
#         driver.close()



In [4]:
# import glob

# path = r'/home/stu/code/dans' # use your path
# all_files = glob.glob(path + "/scraped*.csv")

# li = []

# for filename in all_files:
#     df = pd.read_csv(filename, index_col=None, header=0)
#     li.append(df)

# frame = pd.concat(li, axis=0, ignore_index=True)

In [5]:
# wines = frame
# # And now I'll filter out the product linkes
# wines = wines[wines.URL.str.contains('https://www.danmurphys.com.au/product')]

# wines = wines.replace(r'https://www.danmurphys.com.au/product/DM_', r'', regex=True)
# wines = wines.replace(r'/.*$', r'', regex=True)
# wines = wines.drop_duplicates()
# wines.to_csv("about_to_api.csv")

In [ ]:
# #wines
# #mylist = df['Cleaned'].tolist()
# mylist = wines['URL'].tolist()

# #ylist = mylist[0:100]
# result = []

# for wine in mylist:
#     url = "https://api.danmurphys.com.au/apis/ui/Product/" + str(wine)
#     try:
#         r = requests.get(url).json()
#         rtoo = pd.json_normalize(r["Products"])
#         result.append(rtoo)
#     except:
#         print("No product found")

# my_df = pd.concat(result)
# file_res = "API_results_" + time.strftime("%Y%m%d") + ".csv"
# my_df.to_csv(file_res)
# #pd.json_normalize(flatten(r))




No product found


In [5]:
#file_res = "API_results_" + time.strftime("%Y%m%d") + ".csv"
file_res = "API_results_20210817.csv"
my_df = pd.read_csv(file_res)

## Categories
#my_df["Categories"]
# Willnjust keep 2 levels.
my_df["Categories"] = my_df["Categories"].map(eval, na_action='ignore')
new_df = my_df["Categories"].apply(pd.Series)
my_df["Categories"] = new_df[0].apply(pd.Series).UrlFriendlyName
my_df["Sub_Categories"] = new_df[1].apply(pd.Series).UrlFriendlyName



In [6]:

## Reviews
my_df["Reviews"] = my_df["Reviews"].map(eval,  na_action='ignore')
# Try with first 2 reviews
new_df = my_df["Reviews"].apply(pd.Series)
# First
my_df["Review1_auth"] = new_df[0].apply(pd.Series).author.apply(pd.Series).Value
my_df["Review1_authorcontent"] = new_df[0].apply(pd.Series).authorcontent.apply(pd.Series).Value
my_df["Review1_points"] = new_df[0].apply(pd.Series).points.apply(pd.Series).Value
my_df["Review1_source"] = new_df[0].apply(pd.Series).source.apply(pd.Series).Value
my_df["Review1_text"] = new_df[0].apply(pd.Series).text.apply(pd.Series).Value
my_df["Review1_vintage"] = new_df[0].apply(pd.Series).vintage.apply(pd.Series).Value
# Second
my_df["Review2_auth"] = new_df[1].apply(pd.Series).author.apply(pd.Series).Value
my_df["Review2_authorcontent"] = new_df[1].apply(pd.Series).authorcontent.apply(pd.Series).Value
my_df["Review2_points"] = new_df[1].apply(pd.Series).points.apply(pd.Series).Value
my_df["Review2_source"] = new_df[1].apply(pd.Series).source.apply(pd.Series).Value
my_df["Review2_text"] = new_df[1].apply(pd.Series).text.apply(pd.Series).Value
my_df["Review2_vintage"] = new_df[1].apply(pd.Series).vintage.apply(pd.Series).Value



In [7]:
# Additional details
my_df["AdditionalDetails"] = my_df["AdditionalDetails"].map(eval,  na_action='ignore')

In [6]:
#my_df.to_csv("pre-norm.csv")

In [8]:

full_df = copy.deepcopy(my_df)

In [172]:
avail = full_df[full_df["IsForDelivery"]][["Stockcode"]]

In [140]:
#full_df.to_csv("full.csv")

In [10]:
# Can't use nested lists of JSON objects in pd.json_normalize
my_df = my_df.explode(column="AdditionalDetails").reset_index(drop=True)

In [11]:
#pd.json_normalize(my_df['AdditionalDetails']).pivot(columns='Name')['Value']

In [12]:
# pd.json_normalize expects a list of JSON objects not a DataFrame
#my_df = pd.concat([my_df, pd.json_normalize(my_df['AdditionalDetails'])], axis=1) 

In [13]:
add_df = pd.DataFrame(pd.json_normalize(my_df["AdditionalDetails"]))

In [14]:
del add_df["DisplayName"]

In [15]:
df = pd.concat([my_df,add_df],axis=1)

In [16]:
df = df.pivot(index='Stockcode',columns='Name', values='Value')

In [17]:
df.to_csv("mydf.csv")
df = pd.read_csv("mydf.csv")

In [18]:
#newdf =  pd.concat([full_df,df],axis=1)

In [19]:
#full_df['Stockcode'] = df['Stockcode'].astype(str)

In [20]:
#newdf = full_df.set_index('Stockcode').join(df.set_index('Stockcode'), lsuffix='_full', rsuffix='_new')

In [21]:
newdf = pd.merge(full_df, df, on='Stockcode')

In [22]:
newdf.to_csv("newdf.csv")

In [23]:
newdf = pd.read_csv("newdf.csv")

/home/stu/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (90,91,92,108,114,121,125) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [152]:
list(newdf)

['Unnamed: 0',
 'RecommendedProducts',
 'ProductsInSameOffer',
 'Categories',
 'BackorderMessage',
 'Reviews',
 'DeliveryOptionsInfo',
 'SavedLists',
 'Stockcode',
 'Description',
 'SmallImageFile',
 'MediumImageFile',
 'LargeImageFile',
 'IsOnSpecial',
 'IsMemberSpecial',
 'IsEdrSpecial',
 'QuantityInTrolley',
 'Unit',
 'MinimumQuantity',
 'IsInTrolley',
 'Source',
 'SupplyLimit',
 'PackageSize',
 'IsForCollection',
 'IsForDelivery',
 'IsPurchasable',
 'IsFindMeAvailable',
 'ShouldShowFindMeCta',
 'ProductTags',
 'ProductSashes',
 'UniqueSellingProposition',
 'AgeRestricted',
 'DisplayQuantity',
 'RichDescription',
 'IsFeaturedTag',
 'OverallRating',
 'NumberOfReviews',
 'IsNew',
 'ImageVariants',
 'AdditionalDetails',
 'ParentStockCode',
 'StockOnHand',
 'BackorderStockOnHand',
 'IsPreSale',
 'IsComingSoon',
 'UrlFriendlyName',
 'AvailablePackTypes',
 'IsInWishList',
 'IsInDefaultList',
 'IsDeliveryOnly',
 'GroupedDetails.image',
 'GroupedDetails.workflow',
 'PickupStoreInventoryInfo

In [ ]:
Text to TFIDF
'webdescriptionshort',
'RichDescription',
 

In [24]:
gives = newdf[['Stockcode','webproductname','Prices.singleprice.Value','Prices.promoprice.Value','Prices.promoprice.BeforePromotion','Prices.promoprice.AfterPromotion']]

In [25]:
gives = gives[gives.webproductname.notnull()]

In [26]:
gives = gives[gives['Stockcode'].str.contains("MYSTERY")]

In [27]:
gives = gives[~gives["webproductname"].str.contains("Wraps")]

In [28]:
gives

,Stockcode,webproductname,Prices.singleprice.Value,Prices.promoprice.Value,Prices.promoprice.BeforePromotion,Prices.promoprice.AfterPromotion
727,MYSTERY437,Alkoomi Blackbutt Cabernet Merlot Cabernet Franc,59.99,30.0,59.99,30.0
772,MYSTERY411,Craggy Range Le Sol Syrah 2010,155.99,79.9,155.99,79.9
951,MYSTERY459,Forest Hill Block 5 Caberent Sauvignon,30.00,NaN,NaN,NaN
1214,MYSTERY261,Known Pleasures McLaren Vale Shiraz 2017,59.99,20.0,59.99,20.0
1585,MYSTERY28,Secret Selection McLaren Vale Shiraz Grenache ...,49.99,16.9,49.99,16.9
1774,MYSTERY451,Peos Estate Pinot Noir,8.00,NaN,NaN,NaN
2029,MYSTERY406,Alkoomi Frankland River Shiraz Viognier,21.99,14.9,21.99,14.9
2154,MYSTERY513,*,39.99,15.0,39.99,15.0
2888,MYSTERY378,Peos Estate Cab Sauv,22.99,15.0,22.99,15.0
3046,MYSTERY540,Jim Barry The Benbournie Cabernet Sauvignon 2014,35.88,NaN,NaN,NaN


In [29]:
keep =['Categories',
 'Stockcode',
 'PackageSize',
 'Prices.inanysixprice.Message',
 'Prices.inanysixprice.Value',
 'Sub_Categories',
 'Review1_auth',
 'Review1_points',
 'Review1_source',
 'awardwinner',
 'glutenfree',
 'preservativefree',
 'varietal',
 'webalcoholpercentage',
 'webbottleclosure',
 'webcountryoforigin',
 'webfoodmatch',
 'webisorganic',
 'webisvegan',
 'webliquorsize',
 'webmaincategory',
 'webregionoforigin',
 'webstateoforigin',
 'webtotalreviewcount',
 'webwinebody',
 'webwinestyle']                                                                                                      

In [30]:
keep1 =['Categories',
 'Stockcode',
 'PackageSize',
 'RichDescription',
 'Review1_text',
 'Review2_text',
 'Prices.inanysixprice.Message',
 'Prices.inanysixprice.Value',
 'Sub_Categories',
 'Review1_auth',
 'Review1_points',
 'Review1_source',
 'awardwinner',
 'glutenfree',
 'preservativefree',
 'varietal',
 'webalcoholpercentage',
 'webbottleclosure',
 'webcountryoforigin',
 'webdescriptionshort',
 'webfoodmatch',
 'webisorganic',
 'webisvegan',
 'webliquorsize',
 'webmaincategory',
 'webregionoforigin',
 'webstateoforigin',
 'webtotalreviewcount',
 'webwinebody',
 'webwinestyle']     

In [31]:




#check = pd.json_normalize(my_df["AdditionalDetails"]).pivot(columns='Name')

#check.to_csv("check.csv")

#check['Value']
#my_df["AdditionalDetails"].apply(pd.Series.explode)

########### Would likely want ot save to DB at this tome/.

#In excel: =TEXTJOIN("','",TRUE,A1:CF1)
#keep = ['Categories','BackorderMessage','varietal','DeliveryOptionsInfo','SavedLists','Stockcode','PackageSize','RichDescription','StockOnHand','UrlFriendlyName','IsDeliveryOnly','Prices.inanysixprice.Value','Prices.caseprice.Value','Prices.singleprice.Value','Inventory.availableinventoryqty','Prices.promoprice.Message','Prices.promoprice.Value','Prices.promoprice.PreText','Prices.promoprice.BeforePromotion','Prices.promoprice.AfterPromotion','Sub_Categories','Review1_auth','Review1_authorcontent','Review1_source','Review1_points','Review1_text','Review1_vintage','awardwinner','brewery','corkscoreeligible','countryoforigin','dm_stockcode','image1','image2','standarddrinks','webalcoholpercentage','webaverageproductrating','webbadgescollection','webbottleclosure','webcountryoforigin','webdescriptionshort','webdsvflag','webfoodmatch','webisvegan','weblangtonsclassification','webliquorsize','webmaincategory','webmaxquantity','webminquantity','webpacksizecase','webpacktype','webpresaleenddate','webpresaleflag','webpresalemarketlaunchdate','webpresalemdmmaxqtylimit','webpresaleorderreleasedate','webpresalestartdate','webpresaleusermaxqtylimit','webproductcanbechilled','webproductname','webproductsale','webproducttype','webpromomdmmessage','webpromomessage','webpromomessageenddt','webpromomessagestartdt','webpromosecondmesgenddt','webpromosecondmesgstartdt','webpromosecondmessage','webpromotionalbundle','webregionoforigin','webstateoforigin','webtotalreviewcount','webvideourl','webvintagecurrent','webvintagenote','webwinebody','webwinemaker','webwinestyle']
#keep = ['Categories','BackorderMessage','Description','DeliveryOptionsInfo','SavedLists','Stockcode','PackageSize','StockOnHand','UrlFriendlyName','IsDeliveryOnly','Prices.inanysixprice.Value','Prices.caseprice.Value','Prices.singleprice.Value','Inventory.availableinventoryqty','Prices.promoprice.Message','Prices.promoprice.Value','Prices.promoprice.PreText','Prices.promoprice.BeforePromotion','Prices.promoprice.AfterPromotion','Sub_Categories','Review1_auth','Review1_authorcontent','Review1_source','Review1_points','Review1_text','Review1_vintage','awardwinner','brewery','corkscoreeligible','countryoforigin','dm_stockcode','image1','image2','standarddrinks','webalcoholpercentage','webaverageproductrating','webbadgescollection','webbottleclosure','webcountryoforigin','webdescriptionshort','webdsvflag','webfoodmatch','webisvegan','weblangtonsclassification','webliquorsize','webmaincategory','webmaxquantity','webminquantity','webpacksizecase','webpacktype','webpresaleenddate','webpresaleflag','webpresalemarketlaunchdate','webpresalemdmmaxqtylimit','webpresaleorderreleasedate','webpresalestartdate','webpresaleusermaxqtylimit','webproductcanbechilled','webproductname','webproductsale','webproducttype','webpromomdmmessage','webpromomessage','webpromomessageenddt','webpromomessagestartdt','webpromosecondmesgenddt','webpromosecondmesgstartdt','webpromosecondmessage','webpromotionalbundle','webregionoforigin','webstateoforigin','webtotalreviewcount','webvideourl','webvintagecurrent','webvintagenote','webwinebody','webwinemaker','webwinestyle']
# Timmed version
#### OLD KEEP
#keep = ['Description','Stockcode','PackageSize','Prices.inanysixprice.Value','Prices.caseprice.Value','Prices.singleprice.Value','Prices.promoprice.Message','Prices.promoprice.Value','Prices.promoprice.PreText','Sub_Categories','Review1_auth','Review1_points','Review1_vintage','awardwinner','countryoforigin','standarddrinks','webalcoholpercentage','webaverageproductrating','webbadgescollection','webbottleclosure','webcountryoforigin','webdescriptionshort','webdsvflag','webfoodmatch','webisvegan','weblangtonsclassification','webliquorsize','webmaincategory','webpacksizecase','webpacktype','webregionoforigin','webstateoforigin','webvintagecurrent','webvintagenote','webwinebody','webwinestyle']
# Now Keep them.
kept = newdf[keep]
#kept.to_csv('fri_big_run_kept.csv')
kept.reset_index(drop=True, inplace=True)

In [32]:
kept.to_csv("kept.csv")

In [33]:
#text to TDIF
'webdescriptionshort',
'RichDescription',

('RichDescription',)

In [65]:
#kept = pd.read_csv("kept.csv")

In [34]:
known = kept
known.reset_index(drop=True, inplace=True)

In [35]:
#### ONE HOT ENCODED
##### First I split into numeric and nominal. OHE the nominal
exclude_col = known.select_dtypes(include=np.number).columns.tolist() + ["Stockcode"]
my_df_num = known[exclude_col]
my_df_cat = known.drop(exclude_col, axis=1)
#my_df_cat.to_csv("FIN.csv")
my_df_cat_ohe = pd.get_dummies(my_df_cat)
my_df_ohe = pd.concat([my_df_num,my_df_cat_ohe], axis=1)
my_df_ohe = my_df_ohe.fillna(0)
my_df_ohe = my_df_ohe.replace(np.nan, 0)

# Drop duplicates #TODO check whats better to keep
my_df_ohe = my_df_ohe.loc[:,~my_df_ohe.columns.duplicated()]
# Clean up names
my_df_ohe.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in my_df_ohe.columns.values]

myst  = my_df_ohe[my_df_ohe.Stockcode.str.contains("MYSTERY")]
known  = my_df_ohe[~my_df_ohe.Stockcode.str.contains("MYSTERY")]

myst_nn = myst.drop("Stockcode", axis=1)
known_nn = known.drop("Stockcode", axis=1)


In [36]:

from sklearn.neighbors import NearestNeighbors
# Create the k-NN model using k=5
nn_abs = NearestNeighbors(n_neighbors=1, algorithm='auto')

# Fit it
nn_abs.fit(known_nn)

#distance, matches = nn_abs.kneighbors(myst_nn.iloc[[9]], 2, return_distance=True)
#matches

#known["Stockcode"].iloc[matches[0]]

#myst['Stockcode'].iloc[[9]]
#Now, tokenise each descition.
#MAGIC TIME

#myst.iloc[[9]].to_csv("myst.csv")
#myst_nn.iloc[[9]].to_csv("myst_nn.csv")



#known.iloc[matches[0]].to_csv("known.csv")
#known_nn.iloc[matches[0]].to_csv("known_nn.csv")

results_wine = []

for index in range(len(myst.index)):
    distance, matches = nn_abs.kneighbors(myst_nn.iloc[[index]], 1, return_distance=True)
    results_wine.append(
        {
            'Mystery': "https://www.danmurphys.com.au/product/" + str(myst['Stockcode'].iloc[[index][0]]),
            'Mystery Stockcode': str(myst['Stockcode'].iloc[[index][0]]),
            'Matched': "https://www.danmurphys.com.au/product/" + str(known["Stockcode"].iloc[matches[0][0]]),
            'Matched Stockcode':  str(known["Stockcode"].iloc[matches[0][0]]),
            'Distance': str(distance[0][0])
            
        }
    )


matched = pd.DataFrame(results_wine)

#def create_clickable_id(id):
#    url_template= '''<a href="../../link/to/{id}" target="_blank">{id}</a>'''.format(id=id)
#    return url_template

#matched['Mystery'] = matched['Mystery'].apply(create_clickable_id)
#matched['Matched'] = matched['Matched'].apply(create_clickable_id)

#file_match = "matched_results_" + time.strftime("%Y%m%d") + ".csv"

#matched.to_html("matched.html")



#matched.to_csv("matched_sun_1.csv")

#matches = nn_abs.kneighbors(myst_nn.iloc[[3]], 2, return_distance=False)
#known["Stockcode"].iloc[matches[0]]

#myst['Stockcode'].iloc[[3]]




In [113]:
#matched.to_csv("Sat_pm.csv")

In [181]:
#TODO Now just need to join back with the stockcode, and we've got a working MVP..
promos = newdf[newdf.Stockcode.str.contains("MYSTERY")][['Stockcode','webproductname','Prices.singleprice.Value','Prices.promoprice.Value']]
cutdown = newdf[['Stockcode','Description','producttitle']]

In [182]:
final = pd.merge(matched, promos, left_on='Mystery Stockcode', right_on='Stockcode')

In [183]:
final = pd.merge(final, cutdown, left_on='Matched Stockcode', right_on='Stockcode')

In [185]:
final = pd.merge(final, avail, left_on='Mystery Stockcode', right_on="Stockcode")

In [186]:
final = final[["Mystery","Matched","Distance","webproductname","Prices.singleprice.Value","Prices.promoprice.Value","Description","producttitle"]]

In [187]:
final['Savings'] = final["Prices.singleprice.Value"] - final["Prices.promoprice.Value"]

In [188]:
final["MatchLevel"] = np.where(final['Distance'].astype(float) < 1.8, "Good", "Poor")
final = final.sort_values(['MatchLevel', 'Savings'], ascending=[True, False])

In [189]:
final = final[final.MatchLevel.str.contains("Good")]

In [190]:
def make_clickable(val):
    # target _blank to open new window
    return '<a target="_blank" href="{}">{}</a>'.format(val, val)

final = final.style.format({'Mystery': make_clickable,'Matched': make_clickable,})\
                   .bar(subset=['Savings'], align='mid', color=['#5fba7d'])\
                   .bar(subset=['Savings'], align='mid', color=['#5fba7d'])\
                   .hide_index()\
    
        

In [191]:

#final = final.style.format({'Mystery': make_clickable,'Matched': make_clickable,})
#final = final.style.format({'Matched': make_clickable})

In [44]:
#writing HTML Content
heading = '<h1> Matched wines</h1>'
subheading = '<h3> Results sub header </h3>'
# Using .now() from datetime library to add Time stamp
now = datetime.now()
current_time = now.strftime("%m/%d/%Y %H:%M:%S")
header = '<div class="top">' + heading + subheading +'</div>'
footer = '<div class="bottom"> <h3> This Report has been Generated on'+ current_time +'</h3> </div>'
content = final
# Concating everything to a single string

In [45]:
html = header + content.render() + footer
# Writing the file
with open('report.html','w+') as file:
    file.write(html)
    #file.write(content)
    #file.write(footer)

In [49]:
final.to_html("final")

AttributeError: 'Styler' object has no attribute 'to_html'

In [101]:
final_res = "Matched_results_" + time.strftime("%Y%m%d") + ".html"
f=open(final_res,"w")
f.write(final.render()) # df is the styled dataframe
f.close()
#final.to_html(final_res)

# Now any easy ones just matching descirptions

In [79]:
kept = newdf[keep1]
#kept.to_csv('fri_big_run_kept.csv')
newdf['Review1_text'] = full_df['Review1_text']
newdf['Review2_text'] = full_df['Review2_text']

kept.reset_index(drop=True, inplace=True)
myst  = kept[kept.Stockcode.str.contains("MYSTERY")]
known  = kept[~kept.Stockcode.str.contains("MYSTERY")]


In [81]:
desc_match = pd.merge(myst[myst['RichDescription'].notna()], known, on=['RichDescription'], how='inner')

In [82]:
webdesc_match = pd.merge(myst[myst['webdescriptionshort'].notna()], known, on=['webdescriptionshort'], how='inner')

In [83]:
rev_match = pd.merge(myst[myst['Review1_text'].notna()], known, on=['Review1_text'], how='inner')

In [87]:
rev1_match = pd.merge(myst[myst['Review2_text'].notna()], known, on=['Review2_text'], how='inner')

In [161]:
text_match = desc_match.append(webdesc_match).append(rev_match).append(rev1_match).reset_index()

In [162]:
text_match = text_match[['Stockcode_x','Stockcode_y']]

In [163]:
#TODO Now just need to join back with the stockcode, and we've got a working MVP..
#promos = newdf[newdf.Stockcode.str.contains("MYSTERY")][['Stockcode','webproductname','Prices.singleprice.Value','Prices.promoprice.Value']]
#cutdown = newdf[['Stockcode','Description','producttitle']]

In [164]:
text_match = pd.merge(text_match, newdf, left_on='Stockcode_x', right_on='Stockcode')[['Stockcode_x','Stockcode_y','Description', 'Prices.singleprice.Value','Prices.promoprice.Value']]

In [166]:
text_match = pd.merge(text_match, newdf, left_on='Stockcode_y', right_on='Stockcode')[['Stockcode_x','Stockcode_y','Description_y', 'Prices.singleprice.Value_x','Prices.promoprice.Value_x']]


In [168]:
text_match['Savings'] = text_match["Prices.singleprice.Value_x"] - text_match["Prices.promoprice.Value_x"]

In [169]:
text_match = text_match.sort_values(['Savings'], ascending=[False])

In [170]:
text_match.to_csv("Text_match")

# Now to try and do the NLP Work!

In [36]:
#Text to TFIDF
#'webdescriptionshort',
#'RichDescription',
keep1 =['Stockcode',
 'webdescriptionshort',
'RichDescription']     
#keep1 =[
# 'webdescriptionshort',
#'RichDescription','Review1_text','Review2_text']     


In [78]:
kept = newdf[keep1]
#kept.to_csv('fri_big_run_kept.csv')
kept.reset_index(drop=True, inplace=True)

In [29]:
# Using http://mlbernauer.github.io/R/20160131-document-retrieval-sklearn.html

In [79]:
## Adding in tfidf for description, and then dropping the columns

from sklearn.feature_extraction.text import TfidfVectorizer

#transformer = TfidfVectorizer(max_features=1000, stop_words='english')

#v = TfidfVectorizer(ngram_range=(2, 3))
#x = v.fit_transform(kept['RichDescription'].values.astype('U'))



In [96]:
# Create vectorizer for webdescriptionshort, max_df is set to 0.5, we only want
# to include terms that appear in less tha 50% of the documents (i.e. rare terms)
web_tfidf_vectorizer = TfidfVectorizer(max_features=1000000, use_idf=True,ngram_range=(1, 3))

# Create vectorizer for , RichDescriptions max_df is set to 0.5, we only want 
# to include terms that appear in less than 50% of the documents (i.e. rare terms)
rich_tfidf_vectorizer = TfidfVectorizer(max_features=1000000, use_idf=True,ngram_range=(1, 3))

In [97]:
# Fit Weights for both
tfidf_weights_web = web_tfidf_vectorizer.fit_transform(kept['webdescriptionshort'].values.astype('U'))
tfidf_weights_rich = rich_tfidf_vectorizer.fit_transform(kept['RichDescription'].values.astype('U'))


In [98]:
# Get feature names for Abstract and Title models
tfidf_features_web = web_tfidf_vectorizer.get_feature_names()
tfidf_features_rich = rich_tfidf_vectorizer.get_feature_names()

In [99]:
# Build model to return 5 closest neighbors
from sklearn.neighbors import NearestNeighbors

# Create the k-NN model using k=5
nn_web = NearestNeighbors(n_neighbors=5, algorithm='auto')
nn_rich = NearestNeighbors(n_neighbors=5, algorithm='auto')

# Fit the models to the TF-IDF weights matrix
nn_fitted_web = nn_web.fit(tfidf_weights_web)
nn_fitted_rich = nn_rich.fit(tfidf_weights_rich)

In [100]:
def find_nearest_papers(row, kNNmodel, tfidf_weights, tfidf_features, papers):
    #keywords = get_top_features(row, tfidf_weights, tfidf_features)
    dist,idx = kNNmodel.kneighbors(tfidf_weights[row,:])
    idx = list(idx[0])
    return {'Wines':kept.iloc[idx]}


In [101]:
def get_top_features(rownum, weights, features, top_k=10):
    weight_vec = weights.toarray()[rownum,:]
    top_idx = np.argsort(weight_vec)[::-1][:top_k]
    return [features[i] for i in top_idx]

In [102]:
kept.index[kept['Stockcode'] == 'MYSTERY265']

Int64Index([1115], dtype='int64')

In [106]:
kept.index[kept['Stockcode'] == 'MYSTERY378']

Int64Index([2888], dtype='int64')

In [ ]:
#df.index[df['Stockcode'] == 'MYSTERY265']

In [109]:
#find_nearest_papers(1, nn_fitted_abs, tfidf_weights_abs, tfidf_features_abs, papers)['papers']
find_nearest_papers(1115, nn_fitted_web, tfidf_weights_web, tfidf_features_web, kept)['Wines']

,Stockcode,webdescriptionshort,RichDescription
1115,MYSTERY265,<p>The fruit for this iconic wine is handpicke...,NaN
3045,503063,<p>The fruit for this iconic wine is handpicke...,<p>The fruit for this iconic wine is handpicke...
1895,551372,<p>The fruit for this iconic wine is handpicke...,<p>The fruit for this iconic wine is handpicke...
3866,868188,....,....
1329,55017,The Shiraz grapes for The Struie come from old...,NaN


In [108]:
#find_nearest_papers(1, nn_fitted_abs, tfidf_weights_abs, tfidf_features_abs, papers)['papers']
find_nearest_papers(2888, nn_fitted_rich, tfidf_weights_rich, tfidf_features_rich, kept)['Wines']

,Stockcode,webdescriptionshort,RichDescription
2888,MYSTERY378,This Western Australia Cabernet Sauvignon is m...,Dry Cab Sauv.
3866,868188,....,....
123,86912,Leconfield&apos;s Cabernet Sauvignon is the fl...,A classic Cab Sauv from Leconfield's with swee...
1696,824711,"A rich, smooth Coonawarra Cabernet Sauvignon. ...","<p>A rich, smooth Coonawarra Cab Sauv, Craftsm..."
367,908873,Hollick Cabernet with all it&apos;s bouqet and...,"A classic Coonawarra Cab Sauv, Hollick's bouqu..."


In [ ]:

#my_df_ohe.to_csv("OHE.csv")
#myst_nn = myst_nn.reset_index()
#myst = myst.reset_index()

#known_nn = known_nn.reset_index()
#known = known.reset_index()

##### Now myst
#my_df_num_m = myst[exclude_col]
#my_df_cat_m = myst.drop(exclude_col, axis=1)
#my_df_cat.to_csv("FIN.csv")
#my_df_cat_ohe_m = pd.get_dummies(my_df_cat_m)
#my_df_ohe_m = pd.concat([my_df_num_m,my_df_cat_ohe_m], axis=1)
#my_df_ohe_m = my_df_ohe_m.drop("Stockcode", axis=1)
#my_df_ohe_m = my_df_ohe_m.fillna(0)
#my_df_ohe_m = my_df_ohe_m.replace(np.nan, 0)


from sklearn.neighbors import NearestNeighbors
# Create the k-NN model using k=5
nn_abs = NearestNeighbors(n_neighbors=5, algorithm='auto')


# Checkif we've got NsNS

#
#wines_corr =  my_df_ohe_d.corr(method = "pearson")
#
nn_abs.fit(known_nn)

distance, matches = nn_abs.kneighbors(myst_nn.iloc[[9]], 2, return_distance=True)
#matches

known["Stockcode"].iloc[matches[0]]

myst['Stockcode'].iloc[[9]]
#Now, tokenise each descition.
#MAGIC TIME

myst.iloc[[9]].to_csv("myst.csv")
myst_nn.iloc[[9]].to_csv("myst_nn.csv")



known.iloc[matches[0]].to_csv("known.csv")
known_nn.iloc[matches[0]].to_csv("known_nn.csv")

results_wine = []

for index in range(len(myst.index)):
    distance, matches = nn_abs.kneighbors(myst_nn.iloc[[index]], 1, return_distance=True)
    results_wine.append(
        {
            'Mystery': "https://www.danmurphys.com.au/product/" + str(myst['Stockcode'].iloc[[index][0]]),
            'Matched': "https://www.danmurphys.com.au/product/" + str(known["Stockcode"].iloc[matches[0][0]]),
            'Distance': str(distance[0][0])
        }
    )


matched = pd.DataFrame(results_wine)

#def create_clickable_id(id):
#    url_template= '''<a href="../../link/to/{id}" target="_blank">{id}</a>'''.format(id=id)
#    return url_template

#matched['Mystery'] = matched['Mystery'].apply(create_clickable_id)
#matched['Matched'] = matched['Matched'].apply(create_clickable_id)

matched.to_html("matched.html")

matched.to_csv("matched_sat_1.csv")

matches = nn_abs.kneighbors(myst_nn.iloc[[3]], 2, return_distance=False)
known["Stockcode"].iloc[matches[0]]

myst['Stockcode'].iloc[[3]]




https://www.danmurphys.com.au/product/DM_MYSTERY352/under-wraps-beechworth-sangiovese-2018

https://www.danmurphys.com.au/product/7252


## xgboost

from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X = known_nn
Y = known['Stockcode']

X_test = myst_nn
Y_test = myst['Stockcode']

# Dont need this if already cleaned ohe
#X.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in X.columns.values]


#
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)


# fit model no training data
model = XGBClassifier(verbosity=2)
model.fit(X, Y)

# Check it out now
print(model)


### NB
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(X, Y)
GaussianNB(priors=None)
y_pred = clf.predict(X_test_std)
## Now to predict

y_pred = clf.predict(X_test)
#y_pred holds the predicted label of your test set.
## Finally time to see the accuracy of our estimator.

from sklearn.metrics import accuracy_score
accuracy_score(y_true=y_test, y_pred=y_pred)

pd.DataFrame(y_pred).to_csv("Pred_NB.CSV")

pd.DataFrame(Y_test).to_csv("Myst_NB.CSV")Ansd t